## Calculate features from ICs

- Edge fraction
- High frequency content
- ...
- (features from paper)

In [1]:
import os, psutil
import sys
from datetime import datetime

import numpy as np
import pandas as pd
from nilearn.image import load_img, threshold_img, math_img, resample_to_img
from scipy.ndimage.morphology import binary_erosion
from scipy.signal import periodogram
from os.path import join, pardir
sys.path.append(pardir)
from bids import BIDSLayout
from tqdm import tqdm
import humanize

# plotting
import matplotlib.pyplot as plt
import seaborn as sb
from nilearn.plotting import plot_img
from nilearn.masking import intersect_masks, apply_mask

/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


In [2]:
# Helper function
def get_comps(metainfo_dict):
    """
    ...
    """
    # retrieve files
    mixmat = np.loadtxt(join(metainfo_dict['fullpath'], 'melodic_mix'))
    ica_nii_f = join(metainfo_dict['fullpath'], 'melodic_IC.nii.gz')
    comps_arr = load_img(ica_nii_f).get_fdata()
    return mixmat, comps_arr

# Helper function
def get_masks(metainfo_dict, ds_layout):
    """
    ...
    """
    # retrieve fmriprep/func brainmask file
    brainmask_f_temp = ds_layout.get(
        scope='derivatives',
        return_type='filename',
        subject=metainfo_dict['subject'],
        session=metainfo_dict['session'],
        run=metainfo_dict['run'],
        task=metainfo_dict['task'],
        space=metainfo_dict['space'],
        desc='brain',
        suffix='mask',
        extension='nii.gz'
    )
    #print("brainmask_f_temp", brainmask_f_temp)
    #sys.exit()
    brainmask_f = brainmask_f_temp[0]
    # Not sure if I have correct file here, file from Oli's script is:
    # 'fmriprep/../sub-XX_acq-prescannormalized_rec-pydeface_label-CSF_probseg.nii.gz'
    if metainfo_dict['space'] == 'T1w':
        csf_anat_f_temp = ds_layout.get(
                #scope='fmriprep', -> seems not to work, don't know why
                return_type='filename',
                subject=metainfo_dict['subject'],
                #space=metainfo_dict['space'],
                label='CSF',
                suffix='probseg',
                extension='nii.gz'
        )
        csf_anat_f = csf_anat_f_temp[0]
    else:
        csf_anat_f_temp = ds_layout.get(
                #scope='fmriprep', -> seems not to work, don't know why
                return_type='filename',
                subject=metainfo_dict['subject'],
                space=metainfo_dict['space'],
                label='CSF',
                suffix='probseg',
                extension='nii.gz'
        )
        csf_anat_f = csf_anat_f_temp[0]
    csf_func = threshold_img(
        resample_to_img(csf_anat_f, brainmask_f, interpolation='linear'),
        threshold=1.
    )
    brainmask = load_img(brainmask_f).get_fdata()
    mask_img = math_img('img1 - img2', img1=brainmask_f, img2=csf_func)
    mask_arr = mask_img.get_fdata()
    # worked okayish with erosion iterations=2
    edgefrac_thickness = int(2)
    ero_mask = binary_erosion(mask_arr, iterations=edgefrac_thickness).astype(int)
    edgemask = mask_arr - ero_mask
    return edgemask.astype(bool), brainmask.astype(bool)

# Helper function
def get_gmfiles(metainfo_dict, ds_layout):
    bmask_f = ds_layout.get(
        scope='derivatives',
        return_type='filename',
        subject=metainfo_dict['subject'],
        session=metainfo_dict['session'],
        run=metainfo_dict['run'],
        task=metainfo_dict['task'],
        space=metainfo_dict['space'],
        desc='brain',
        suffix='mask',
        extension='nii.gz'
    )
    aseg_f = ds_layout.get(
        scope='derivatives',
        return_type='filename',
        subject=metainfo_dict['subject'],
        session=metainfo_dict['session'],
        run=metainfo_dict['run'],
        task=metainfo_dict['task'],
        space=metainfo_dict['space'],
        desc='aseg',
        suffix='dseg',
        extension='nii.gz'
    )
    # aseg2gm
    gm_left = math_img('img == 3', img=aseg_f)
    gm_right = math_img('img == 42', img=aseg_f)
    gm = intersect_masks([gm_left, gm_right], threshold=0, connected=False)
    return comp_f, bmask_f, gm

# Edge fraction
def calc_edgefrac(comp_arr, edgemask, brainmask):
    return np.absolute(comp_arr[edgemask]).sum() / np.absolute(comp_arr[brainmask]).sum()

# High frequency content
def calc_hfc(timeseries, tr=1.5):
    """
    Calculate high frequency content for time series data.
    Tr can generally mean sampling rate in seconds.
    """
    nf = (1. / tr) * .5  # nyquist
    freqs, power = periodogram(timeseries, fs=1. / tr)
    relcumsum = np.cumsum(power) / power.sum()
    freqind = np.argmin(np.absolute(relcumsum - .5))
    hfc = freqs[freqind] / nf
    return hfc

# Gray matter proportion
def calc_gm_prop(comp_i, bmask_f, gm):
    """
    Calculate the proportion of the (active) gray matter voxels of a volume
    compared to the whole number of gray voxels in the brain.
    """
    comp_f = ds_layout.get(
        scope='melodic',
        return_type='filename',
        suffix=f'zstat{comp_i}',
        extension='nii.gz'
    )
    # count significant voxels in gray matter
    comp_gm = apply_mask(comp_f, gm)
    nsig_gm = np.sum(comp_gm > 0.)
    # count significant voxels in whole brain
    comp_brain = apply_mask(comp_f, bmask_f)
    nsig_brain = np.sum(comp_brain > 0.)
    # return ratio
    gm_prop = nsig_gm / nsig_brain
    return gm_prop

In [ ]:
comp_f = "/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/melodic/sub-03/sub-03_ses-None_task-effort_run-None_space-T1w-melodic/stats/thresh_zstat100.nii.gz"
aseg_f = '/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-03/func/sub-03_task-effort_space-T1w_desc-aseg_dseg.nii.gz'
bmask_f = '/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-03/func/sub-03_task-effort_space-T1w_desc-brain_mask.nii.gz'

In [3]:
# Main function
def calculate_features(bidsdata_dir):
    """Get dict with calculated features for each melodic run."""
    melodic_base_dir = join(bidsdata_dir, 'derivatives', 'melodic')
    print("creating BIDS layout ...")
    ds_layout = BIDSLayout(bidsdata_dir, derivatives=True)
    # TODO: 'melodic' needs dataset_description.json or melodic_entities is empty
    print("get melodic directory names ...")
    melodic_entities = ds_layout.get(scope='melodic', return_type='filename', suffix='IC', extension='nii.gz')
    # Sanity check
    print("[Sanity check] melodic_entities length: ", len(melodic_entities))
    results_dicts = []
    #i = 1 # iteration counter
    for entity in tqdm(melodic_entities, desc=f'iterating over runs'):
        # Cumbersome workaround to get correct filenames (TODO: better filenaming with DataSink!)
        #print("**ITERATION** ", i)
        #j = 1 # iteration counter
        melodic_dir_split = entity.split('/')
        dir_name = melodic_dir_split[-2]
        metainfo_split = dir_name.split('_')
        # Get substring if present
        subject = [s[4:] for s in metainfo_split if "sub-" in s]
        session = [s[4:] for s in metainfo_split if "ses-" in s]
        task = [s[5:] for s in metainfo_split if "task-" in s]
        run = [s[4:] for s in metainfo_split if "run-" in s]
        space = [s[6:] for s in metainfo_split if "space-" in s]
        # Put substring in dict
        metainfo_dict = {
            'subject': None if subject in ([], ['None']) else subject[0],
            'session': None if session in ([], ['None']) else session[0],
            'task': None if task in ([], ['None']) else task[0],
            'run': None if run in ([], ['None']) else run[0],
            'space': None if space in ([], ['None']) else space[0],
            'directory': dir_name,
            'fullpath':'/'.join(melodic_dir_split[:-1])
        }
        #i += 1
        mixmat, comps_arr = get_comps(metainfo_dict)
        edgemask, brainmask = get_masks(metainfo_dict, ds_layout)
        #bmask_f, gm = get_gmfiles(metainfo_dict, ds_layout)

        #print("comp_f", comp_f)
        #print("gm", gm)
        #print("bmask_f", bmask_f)
        #print("--------------------------")
            
        for comp_i in range(mixmat.shape[-1]):
            results_dict = metainfo_dict
            #print("***INSIDE ITERATION*** ", j)
            #sys.exit()
            comp_arr = comps_arr[:, :, :, comp_i]
            comp_ts = mixmat[:, comp_i]
            # get IC stat file for gm prop
            #comp_f = get_zstat(comp_i)
            #print("comp_arr: ", comp_arr.shape)
            #print("edgemask: ", edgemask.shape)
            #print("brainmask: ", brainmask.shape)
            # Calculate edge fraction
            results_dict['edgefrac'] = calc_edgefrac(comp_arr, edgemask, brainmask)
            # Calculate high frequency content
            results_dict['hfc'] = calc_hfc(comp_ts)
            # Calculate gray matter proportion
            #results_dict['gmp'] = calc_gm_prop(comp_f, bmask_f, gm)
            results_dicts.append(results_dict)
            #j += 1
    return results_dicts
    

In [4]:
# Hardcoded paths 1/2
#'/LOCAL/jzerbe/faces_vs_houses/ds002938' #'/LOCAL/jzerbe/emotion_category/ds003548'
bidsdata_dir = '/LOCAL/jzerbe/auditory_vs_visual/ds003717' 
base_dir = '/LOCAL/jzerbe/auditory_vs_visual' #'/LOCAL/jzerbe/emotion_category/' # not needed?

In [5]:
# Start
start=datetime.now()
process = psutil.Process()

# Run
feature_results = calculate_features(bidsdata_dir)

# End
print("Done")
print("Estimated Memory Usage: ", humanize.naturalsize(process.memory_info().rss))  # in bytes 
print ("Estimated Time: ", datetime.now()-start)

creating BIDS layout ...
get melodic directory names ...
[Sanity check] melodic_entities length:  288


iterating over runs: 100%|██████████| 288/288 [51:38<00:00, 10.76s/it]

Done
Estimated Memory Usage:  1.4 GB
Estimated Time:  0:54:47.887626


In [ ]:
import os, psutil
from datetime import datetime
start=datetime.now()

process = psutil.Process()
melodic_base_dir = join(bidsdata_dir, 'derivatives', 'melodic')
print("creating BIDS layout ...")
layout_test = BIDSLayout(bidsdata_dir, derivatives=True)
print("Done")
print("Estimated Memory Usage: ", humanize.naturalsize(process.memory_info().rss))  # in bytes 
print ("Estimated Time: ", datetime.now()-start)

todo:
- df für auditory neu berechnen -> separates results dict jedes mal neu machen und werte anfügen
- gm prop auch über ICs laufen lassen (inner loop), dabei z..100 files nehmen zum loopen (muss in der einen helper function auch angepasst werden, dass da immer die neueste ausgegeben wird)
- plots sind so ok (muss keine scatter plots haben)
- anderes feature noch berechnen und mit einbauen

## Ideen für features
- scanner noise (rekonstruieren)
- augenbewegung/blinzeln?
- explore more masking options (csf mask, ..)
- explore hfc variations (1/3, median, certain frequencies) -> maybe find info in hand-labeling paper
- check out sub-01_task-1back_desc-confounds_timeseries.tsv

## Visualization

In [7]:
# Put results in dataframe
# sort after sapces (T1w, MNI)
results_df_all = pd.DataFrame(feature_results)
# exclude resting from dataset
#results_df = results_df_all[results_df_all['task'] != 'rest']
#results_df

In [8]:
results_df_all.to_csv('/LOCAL/jzerbe/code/ICA-fMRI/results/df_features_auditory-vs-visual.csv')

In [ ]:
df_test = pd.read_csv(r'/LOCAL/jzerbe/code/ICA-fMRI/results/df_features_faces-vs-houses.csv')
df_test

In [ ]:
results_df['space'].unique()

In [ ]:
# Hardcoded paths 2/2
fig_path = '/LOCAL/jzerbe/code/ICA-fMRI/figures/'
fig_ds = 'faces-vs-houses' #'emotion-category' # 'faces-vs-houses'
fig_space = 'T1w'
subplot_spaces = results_df[results_df['space'] == fig_space]

In [ ]:
# Visualize hfc
hfc_plt = plt.hist(subplot_spaces['hfc'], bins=70)
plt.title(f'HFC | {fig_ds} | {fig_space}')
plt.savefig(fig_path + f'{fig_ds}_hfc_{fig_space}' + '.jpg')

In [ ]:
# Seaborn density plot
sb.kdeplot(subplot_spaces['hfc'], color='skyblue', fill=True, alpha=.5, linewidth=1.5)
plt.title(f'HFC | {fig_ds} | {fig_space}')
plt.savefig(fig_path + f'{fig_ds}_hfc_{fig_space}_density' + '.jpg')

In [ ]:
# Visualize edge fraction
edgefrac_plt = plt.hist(subplot_spaces['edgefrac'], bins=100)
plt.title(f'Edge Fraction | {fig_ds} | {fig_space}')
plt.savefig(fig_path + f'{fig_ds}_edgefrac_{fig_space}' + '.jpg')

In [ ]:
# Seaborn density plot
sb.kdeplot(subplot_spaces['edgefrac'], color='skyblue', fill=True, alpha=.5, linewidth=1.5)
plt.title(f'Edge Fraction | {fig_ds} | {fig_space}')
plt.savefig(fig_path + f'{fig_ds}_edgefrac_{fig_space}_density' + '.jpg')